In [1]:
import numpy as np
import tensorflow as tf

In [2]:
N_SAMPLES = 5
NUM_THREADS = 4
k_batchsize = 0  #yes, can be greater than num samples!
                 #gotta figure out for yourself how many times you want to see the same data

# Generating some simple data
# create 1000 random samples, each is a 1D array from the normal distribution (10, 1)
# each vector has 4 dimensions in this example
data = 10 * np.random.randn(N_SAMPLES, 4) + 1
# create 1000 random labels of 0 and 1
target = np.random.randint(0, 2, size=N_SAMPLES)

# Create queue types specified for data and target, shapes specified (4D and scalar)
queue = tf.FIFOQueue(capacity=50, dtypes=[tf.float32, tf.int32], shapes=[[4], []])

enqueue_op = queue.enqueue_many([data, target])
#=======================================================
if (k_batchsize > 0) :
    dequeue_op = queue.dequeue_many(k_batchsize) #batch_size at a time, in a list
else :
    dequeue_op = queue.dequeue() #one at a time, not in a list
#=======================================================

# create NUM_THREADS to do enqueue
qr = tf.train.QueueRunner(queue, [enqueue_op] * NUM_THREADS)


In [3]:
with tf.Session() as sess:
    # Create a coordinator, launch the queue runner threads.
    coord = tf.train.Coordinator()
    enqueue_threads = qr.create_threads(sess, coord=coord, start=True)
    
    for step in xrange(2): 
        if coord.should_stop():
            break
        data_batch, label_batch = sess.run(dequeue_op)
        print('------  step: ' + str(step))
        print('data_batch tf.shape is ' + str(tf.shape(data_batch))) 
        if (k_batchsize > 0) :
            print('batch size is ' + str(data_batch.shape[0])) # (rows, colums)
            print('data size is ' + str(data_batch.shape[1])) #the length of a column
        else:
            print('data size is ' + str(data_batch.shape[0]))
        print label_batch
        print data_batch
        
    coord.request_stop()
    coord.join(enqueue_threads)

------  step: 0
data_batch tf.shape is Tensor("Shape:0", shape=(1,), dtype=int32)
data size is 4
1
[ 8.20465088 -4.16747808  1.97255611 -6.28378201]
------  step: 1
data_batch tf.shape is Tensor("Shape_1:0", shape=(1,), dtype=int32)
data size is 4
1
[ -9.21293449  -1.14715385  -7.73651648  13.16541767]
